<a href="https://colab.research.google.com/github/anandhc6/Assignment-3/blob/main/Finetune_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Finetuning GPT2 using HuggingFace and Tensorflow**

In this colab notebook we set up a simple outline of how you can use Huggingface to fine tune a gpt2 model on finance titles to generate new possible headlines. This notebook uses the hugginface finefuning scripts and then uses the TensorFlow version of the genreated models.

First begin setup by cloning transformers repo. We need to store the training script locally since there isnt an easier way to train tf based gpt2 models as far as I can see.

In [ ]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 93572, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 93572 (delta 93), reused 101 (delta 58), pack-reused 93429
Receiving objects: 100% (93572/93572), 85.83 MiB | 11.45 MiB/s, done.
Resolving deltas: 100% (68602/68602), done.


# **Finetuning GPT2 using HuggingFace and Tensorflow**

In this colab notebook we set up a simple outline of how you can use Huggingface to fine tune a gpt2 model on finance titles to generate new possible headlines. This notebook uses the hugginface finefuning scripts and then uses the TensorFlow version of the genreated models.

First begin setup by cloning transformers repo. We need to store the training script locally since there isnt an easier way to train tf based gpt2 models as far as I can see.

In [1]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 93889, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 93889 (delta 7), reused 3 (delta 0), pack-reused 93871
Receiving objects: 100% (93889/93889), 85.93 MiB | 11.98 MiB/s, done.
Resolving deltas: 100% (68889/68889), done.


In [2]:
# Clone should now be in the machine
!ls

sample_data  transformers


Check to see what gpu we were granted. For Colab Pro it will vary between a Tesla V100 or P100. For normal colab it should be a k80

In [3]:
!nvidia-smi

Tue May 10 06:13:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Change directory location to be in the examples folder and then install any requirements

In [4]:
import os
os.chdir("/content/transformers")
os.chdir("./examples/tensorflow/language-modeling")
!ls

README.md  requirements.txt  run_clm.py  run_mlm.py


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 325 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 93.3 MB/s 
     |████████████████████████████████| 1.1 MB 75.7 MB/s 
     |████████████████████████████████| 136 kB 75.1 MB/s 
     |████████████████████████████████| 212 kB 84.5 MB/s 
     |████████████████████████████████| 77 kB 8.3 MB/s 
     |████████████████████████████████| 127 kB 81.1 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 98.6 MB/s 
     |████████████████████████████████| 144 kB 88.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.

In [6]:
!ls

README.md  requirements.txt  run_clm.py  run_mlm.py


In [7]:
!pip install pyarrow --upgrade

     |████████████████████████████████| 29.3 MB 1.2 MB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [8]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./tensorflow/language-modeling")

In [9]:
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+https://github.com/huggingface/transformers/

  Cloning https://github.com/huggingface/transformers/ to /tmp/pip-req-build-03bevd1u
  Running command git clone -q https://github.com/huggingface/transformers/ /tmp/pip-req-build-03bevd1u
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 4.1 MB/s 
     |████████████████████████████████| 596 kB 88.2 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4115037 sha256=270d2306c6458abce7c7d649b6028ed20aba31bc78a608b6aa76026a4f97afc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-ke_4c17t/wheels/fb/1b/91/0fcf504c386d427d65bbaf663eadf8e18cbf9795394ed7050d
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Set up data from a text file in the format <|title|> some data <|endoftext|> and split into training and eval sets.

In [12]:
#Load Data
from sklearn.model_selection import train_test_split

with open('/content/total_data (1).txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
with open('/content/training.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('/content/evaluation.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

The script below will fine tune GPT2 on your text data that you setup above. This training step will take anywhre from tens of minutes to hours depending on how large your training set is, how many epochs you intend to train on, and if you are using colab or colab pro. We utilize mixed precision in this model to shave off some training time. For a large data set I was using for another experiment it saved us over 30 mins in training time.

In [13]:
!python run_clm.py \
--model_name_or_path distilgpt2 \
--train_file "/content/training.txt" \
--do_train \
--validation_file "/content/evaluation.txt" \
--do_eval \
--per_gpu_train_batch_size 128 \
--save_steps -1 \
--num_train_epochs 15 \
--fp16 \
--output_dir="/content/output_final_1"
#--model_type gpt2-medium \

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/util/structure.py", line 22, in <module>
    from tensorflow.python.data.util import nest
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/util/nest.py", line 36, in <module>
    from tensorflow.python.framework import sparse_tensor as _sparse_tensor
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/sparse_tensor.py", line 24, in <module>
    from tensorflow.python.framework import constant_op
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/constant_op.py", line 25, in <module>
    from tensorflow.python.eager import execute
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager

# **Using the model**
Next lets take our model we just trained and use it to generate some text! We will import the Tensorflow version of the gpt2 language model and set the from_pt flag to True. Then we load a pretrained tokenizer from huggingface. This may take some time to download the tokenizer data.

In [ ]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("/content/output_final_1", from_pt=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/output_final_1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Encoding sample text is now extremely simple using the pretrained tokenizer.

In [ ]:
input_ids = tokenizer.encode("I love Deep Learning", return_tensors='tf')

In [ ]:
# the tf tensor object
input_ids[0]

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([   40,  1842, 10766, 18252], dtype=int32)>

Next we will use the model to generate the text from our input sample. The parameters I used are based on trail and error from playing around with the huggingface tutorial, https://huggingface.co/blog/how-to-generate, which really goes into great detail on how to go about finding the best parameters for generating text. As well they dive into really good information on what each parameter does and how they play into one another.

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=500,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)
beam_output = generated_text_samples
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: I love Deep Learning, can't you see?

1: I love Deep Learning, but I need to talk again

2: I love Deep Learning

3: I love Deep Learning

4: I love Deep Learning



In [ ]:
#Print output for each sequence generated above
beam_output = generated_text_samples
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

0: I love Deep Learning, can't you see?

1: I love Deep Learning, but I need to talk again

2: I love Deep Learning

3: I love Deep Learning

4: I love Deep Learning



# **Conclusion**
And there you have it, a simple end to end outline on how you can use Colab, Huggingface, and Tensorflow to train and generate new text data using GPT-2. There is a lot of playing around with hyperparameters in the generate phase but given enough tweaking and time you can usually find something that works well with your data and task. I found that even with the larger GPT-2 model and more examples, it could still repeat itself a bit so something you have to generate a large number of sequences before you get a set that you like. Even OpenAI made note of this in their initial results for GPT-2 so if at first it doesnt generate what you want keep trying and playing with the parameters!

One tip I did notice was that if you do not setup your examples with a start token, then you run into the issue of repeated phrases more easily. Given more data that might be less of a problem but I ran into that a lot before putting in the start token of <|title|> in my exmaples. This start token also has the added benefit of giving you a generic starting point in the text generation so that each run is mostly unique from the last run if you do not care about having a specific prompt.

# **Finetuning GPT2 using HuggingFace and Tensorflow**

In this colab notebook we set up a simple outline of how you can use Huggingface to fine tune a gpt2 model on finance titles to generate new possible headlines. This notebook uses the hugginface finefuning scripts and then uses the TensorFlow version of the genreated models.

First begin setup by cloning transformers repo. We need to store the training script locally since there isnt an easier way to train tf based gpt2 models as far as I can see.

In [ ]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 93271, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 93271 (delta 76), reused 131 (delta 51), pack-reused 93090
Receiving objects: 100% (93271/93271), 85.76 MiB | 22.00 MiB/s, done.
Resolving deltas: 100% (68337/68337), done.


In [ ]:
# Clone should now be in the machine
!ls

sample_data  transformers


Check to see what gpu we were granted. For Colab Pro it will vary between a Tesla V100 or P100. For normal colab it should be a k80

In [ ]:
!nvidia-smi

Wed May  4 18:04:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Change directory location to be in the examples folder and then install any requirements

In [ ]:
import os
os.chdir("transformers")
os.chdir("./examples/pytorch/language-modeling")
!ls

README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 79 kB 5.3 MB/s 
     |████████████████████████████████| 325 kB 36.1 MB/s 
     |████████████████████████████████| 1.2 MB 47.6 MB/s 
     |████████████████████████████████| 136 kB 55.6 MB/s 
     |████████████████████████████████| 212 kB 53.6 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 1.1 MB 42.1 MB/s 
     |████████████████████████████████| 127 kB 46.7 MB/s 
     |████████████████████████████████| 271 kB 48.1 MB/s 
     |████████████████████████████████| 144 kB 50.3 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium

In [ ]:
!ls

README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [ ]:
!pip install pyarrow --upgrade

     |████████████████████████████████| 26.7 MB 1.5 MB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [ ]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./pytorch/language-modeling")

In [ ]:
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+https://github.com/huggingface/transformers/

  Cloning https://github.com/huggingface/transformers/ to /tmp/pip-req-build-xle3kamb
  Running command git clone -q https://github.com/huggingface/transformers/ /tmp/pip-req-build-xle3kamb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 12.9 MB/s 
     |████████████████████████████████| 596 kB 15.7 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4099889 sha256=c966d13a7bed73941aa22a00e6880588bda4aeb0c829b05f5b7a30512dbc93ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-mpc8qmxy/wheels/fb/1b/91/0fcf504c386d427d65bbaf663eadf8e18cbf9795394ed7050d
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Set up data from a text file in the format <|title|> some data <|endoftext|> and split into training and eval sets.

In [ ]:
"""
Now load the data line by line
"""
from sklearn.model_selection import train_test_split

with open('/content/gdrive/MyDrive/total_data.txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('/content/train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('/content/eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:168346
Evaluation size: 18706


The script below will fine tune GPT2 on your text data that you setup above. This training step will take anywhre from tens of minutes to hours depending on how large your training set is, how many epochs you intend to train on, and if you are using colab or colab pro. We utilize mixed precision in this model to shave off some training time. For a large data set I was using for another experiment it saved us over 30 mins in training time.

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "/content/train_tmp.txt" \
--do_train \
--validation_file "/content/eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="/content/output_dataset_1"

05/04/2022 18:08:03 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
05/04/2022 18:08:03 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN

# **Using the model**
Next lets take our model we just trained and use it to generate some text! We will import the Tensorflow version of the gpt2 language model and set the from_pt flag to True. Then we load a pretrained tokenizer from huggingface. This may take some time to download the tokenizer data.

In [ ]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("/content/output_data", from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Encoding sample text is now extremely simple using the pretrained tokenizer.

In [ ]:
input_ids = tokenizer.encode("I love Deep Learning", return_tensors='tf')

In [ ]:
# the tf tensor object
input_ids[0]

Next we will use the model to generate the text from our input sample. The parameters I used are based on trail and error from playing around with the huggingface tutorial, https://huggingface.co/blog/how-to-generate, which really goes into great detail on how to go about finding the best parameters for generating text. As well they dive into really good information on what each parameter does and how they play into one another.

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=150,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

In [ ]:
#Print output for each sequence generated above
beam_output = generated_text_samples
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

# **Conclusion**
And there you have it, a simple end to end outline on how you can use Colab, Huggingface, and Tensorflow to train and generate new text data using GPT-2. There is a lot of playing around with hyperparameters in the generate phase but given enough tweaking and time you can usually find something that works well with your data and task. I found that even with the larger GPT-2 model and more examples, it could still repeat itself a bit so something you have to generate a large number of sequences before you get a set that you like. Even OpenAI made note of this in their initial results for GPT-2 so if at first it doesnt generate what you want keep trying and playing with the parameters!

One tip I did notice was that if you do not setup your examples with a start token, then you run into the issue of repeated phrases more easily. Given more data that might be less of a problem but I ran into that a lot before putting in the start token of <|title|> in my exmaples. This start token also has the added benefit of giving you a generic starting point in the text generation so that each run is mostly unique from the last run if you do not care about having a specific prompt.

In [ ]:
# Clone should now be in the machine
!ls

sample_data  transformers


Check to see what gpu we were granted. For Colab Pro it will vary between a Tesla V100 or P100. For normal colab it should be a k80

In [ ]:
!nvidia-smi

Wed May  4 18:04:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Change directory location to be in the examples folder and then install any requirements

In [ ]:
import os
os.chdir("transformers")
os.chdir("./examples/pytorch/language-modeling")
!ls

README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 79 kB 5.3 MB/s 
     |████████████████████████████████| 325 kB 36.1 MB/s 
     |████████████████████████████████| 1.2 MB 47.6 MB/s 
     |████████████████████████████████| 136 kB 55.6 MB/s 
     |████████████████████████████████| 212 kB 53.6 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 1.1 MB 42.1 MB/s 
     |████████████████████████████████| 127 kB 46.7 MB/s 
     |████████████████████████████████| 271 kB 48.1 MB/s 
     |████████████████████████████████| 144 kB 50.3 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium

In [ ]:
!ls

README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [ ]:
!pip install pyarrow --upgrade

     |████████████████████████████████| 26.7 MB 1.5 MB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [ ]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./pytorch/language-modeling")

In [ ]:
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+https://github.com/huggingface/transformers/

  Cloning https://github.com/huggingface/transformers/ to /tmp/pip-req-build-xle3kamb
  Running command git clone -q https://github.com/huggingface/transformers/ /tmp/pip-req-build-xle3kamb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 12.9 MB/s 
     |████████████████████████████████| 596 kB 15.7 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4099889 sha256=c966d13a7bed73941aa22a00e6880588bda4aeb0c829b05f5b7a30512dbc93ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-mpc8qmxy/wheels/fb/1b/91/0fcf504c386d427d65bbaf663eadf8e18cbf9795394ed7050d
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Set up data from a text file in the format <|title|> some data <|endoftext|> and split into training and eval sets.

In [ ]:
"""
Now load the data line by line
"""
from sklearn.model_selection import train_test_split

with open('/content/gdrive/MyDrive/total_data.txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('/content/train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('/content/eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:168346
Evaluation size: 18706


The script below will fine tune GPT2 on your text data that you setup above. This training step will take anywhre from tens of minutes to hours depending on how large your training set is, how many epochs you intend to train on, and if you are using colab or colab pro. We utilize mixed precision in this model to shave off some training time. For a large data set I was using for another experiment it saved us over 30 mins in training time.

In [ ]:
!python run_clm.py \
--model_type gpt2-medium \
--model_name_or_path gpt2-medium \
--train_file "/content/train_tmp.txt" \
--do_train \
--validation_file "/content/eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="/content/output_dataset_1"

05/04/2022 18:08:03 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
05/04/2022 18:08:03 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN

# **Using the model**
Next lets take our model we just trained and use it to generate some text! We will import the Tensorflow version of the gpt2 language model and set the from_pt flag to True. Then we load a pretrained tokenizer from huggingface. This may take some time to download the tokenizer data.

In [ ]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("/content/output_data", from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Encoding sample text is now extremely simple using the pretrained tokenizer.

In [ ]:
input_ids = tokenizer.encode("I love Deep Learning", return_tensors='tf')

In [ ]:
# the tf tensor object
input_ids[0]

Next we will use the model to generate the text from our input sample. The parameters I used are based on trail and error from playing around with the huggingface tutorial, https://huggingface.co/blog/how-to-generate, which really goes into great detail on how to go about finding the best parameters for generating text. As well they dive into really good information on what each parameter does and how they play into one another.

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=150,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

In [ ]:
#Print output for each sequence generated above
beam_output = generated_text_samples
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

# **Conclusion**
And there you have it, a simple end to end outline on how you can use Colab, Huggingface, and Tensorflow to train and generate new text data using GPT-2. There is a lot of playing around with hyperparameters in the generate phase but given enough tweaking and time you can usually find something that works well with your data and task. I found that even with the larger GPT-2 model and more examples, it could still repeat itself a bit so something you have to generate a large number of sequences before you get a set that you like. Even OpenAI made note of this in their initial results for GPT-2 so if at first it doesnt generate what you want keep trying and playing with the parameters!

One tip I did notice was that if you do not setup your examples with a start token, then you run into the issue of repeated phrases more easily. Given more data that might be less of a problem but I ran into that a lot before putting in the start token of <|title|> in my exmaples. This start token also has the added benefit of giving you a generic starting point in the text generation so that each run is mostly unique from the last run if you do not care about having a specific prompt.